In [ ]:
from dcicutils import ff_utils

# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

my_env = 'data'
my_auth = ff_utils.get_authentication_with_server({}, ff_env = my_env)

# find all runs from 4dn lab without a contributing lab
q = "/search/?lab.uuid=828cd4fe-ebb0-4b36-a94a-d2e3a36cc989" + \
    "&run_status=complete&type=WorkflowRunAwsem&contributing_labs.uuid=No+value" + \
    "&limit=all"

all_awsem_resp = ff_utils.search_metadata(q, key=my_auth)
print(len(all_awsem_resp))

In [ ]:
counter = 0
for a_resp in all_awsem_resp: 
    counter += 1
    if counter % 100 == 0:
        print(counter)
    # find the identity
    input_files = a_resp['input_files']
    my_lab = ''
    for an_input in input_files:
        # if this wfr ran on processed files
        if an_input['value']['@id'].startswith('/files-processed/'):
            input_resp = ff_utils.get_metadata(an_input['value']['@id'], key=my_auth)
            my_lab = input_resp['lab']['@id']
            if my_lab == '/labs/4dn-dcic-lab/':
                my_lab = input_resp['contributing_labs'][0]['@id']
            break
        # if this wfr ran on raw files
        elif an_input['value']['@id'].startswith('/files-fastq/'):
            input_resp = ff_utils.get_metadata(an_input['value']['@id'], key=my_auth)
            my_lab = input_resp['lab']['@id']
            break
            
    if my_lab == '':
        # some submitted files that gone through md5 and other weird ones
        print(a_resp['uuid'], 'did not have a valid input')
        continue
 
    if my_lab == '/labs/4dn-dcic-lab/':
        print(a_resp['uuid'],'is from 4dn')
        continue
    
    # add contributing lab to wfr
    ff_utils.patch_metadata({'contributing_labs':[my_lab]},obj_id=a_resp['uuid'], key=my_auth)
    
    # check if there are any qcs from the wfr
    qcs = a_resp.get('output_quality_metrics')
    if qcs:
        qc_uuid = qcs[0]['value']['uuid']
        ff_utils.patch_metadata({'contributing_labs':[my_lab]},obj_id=qc_uuid, key=my_auth)
